In [1]:
import sys
# sys.path.append('/home/ubuntu/Workspace/utils/FishFeeding')
from utils.video_preprocess import resize_video

In [1]:
import cv2
cap = cv2.VideoCapture('/Users/liuziyi/Desktop/220731_170753__220731_173752_clip_2.mp4')
# for i in range(10):
while(True):
    check, img = cap.read()
    # print(frame)
    # print(check)
    if check:
        # hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        # cv2.imshow("h", hsv[:, :, 0])
        # cv2.imshow("s", hsv[:, :, 1])
        # cv2.imshow("v", hsv[:, :, 2])
        # yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
        # cv2.imshow("y", yuv[:, :, 0])
        # cv2.imshow("u", yuv[:, :, 1])
        # cv2.imshow("v", yuv[:, :, 2])
        # lab = cv2.cvtColor(img, cv2.COLOR_BGR2Lab)
        # cv2.imshow("l", lab[:, :, 0])
        # cv2.imshow("a", lab[:, :, 1])
        # cv2.imshow("b", lab[:, :, 2])

        # cv2.imshow("u - s", yuv[:, :, 1] - hsv[:, :, 1])
        cv2.imshow("cropped", img[100:700, :, 0])
        key = cv2.waitKey(30)
        if key >= 0:
            cv2.waitKey(0)
    else:
        break
    # if cv2.waitKey(75) & 0xFF == ord('q'):
    #     break

# Close the window / Release webcam
cap.release()
  
# After we release our webcam, we also release the out-out.release() 
  
# De-allocate any associated memory usage 
cv2.destroyAllWindows()

In [3]:
# Close the window / Release webcam
cap.release()
  
# After we release our webcam, we also release the out-out.release() 
  
# De-allocate any associated memory usage 
cv2.destroyAllWindows()

: 